In [1]:
#@title Install Ithaca, download the checkpoint
!pip install -q git+https://github.com/MariaSchoinaki/ithaca || echo "*** FAILED TO INSTALL ITHACA ***"
!curl --output checkpoint.pkl https://storage.googleapis.com/ithaca-resources/models/checkpoint_v1.pkl


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


^C


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0  192M    0  8434    0     0   9094      0  6:10:40 --:--:--  6:10:40  9117
  0  192M    0  702k    0     0   388k      0  0:08:28  0:00:01  0:08:27  389k
  0  192M    0 1566k    0     0   558k      0  0:05:53  0:00:02  0:05:51  559k
  1  192M    1 3440k    0     0   904k      0  0:03:38  0:00:03  0:03:35  905k
  2  192M    2 5229k    0     0  1088k      0  0:03:01  0:00:04  0:02:57 1088k
  3  192M    3 7295k    0     0  1256k      0  0:02:37  0:00:05  0:02:32 1493k
  4  192M    4 9132k    0     0  1341k      0  0:02:27  0:00:06  0:02:21 1685k
  5  192M    5 10.0M    0     0  1318k      0  0:02:29  0:00:07  0:02:22 1744k
  6  192M    6 12.5M    0     0  1454k      0  0:02:15  0:00:08  0:02:07 1871k
  7  192M    7 14.4M    0     0  1514k      0  0:02

In [ ]:
#@title Imports
from IPython.core.display import HTML, display

try:
  import ithaca
  import flax
except ModuleNotFoundError:
  display(HTML('<h1><font color="#f00">Failed to import ithaca. Did installation fail above?</font></h1>'))
  raise

import functools

from flax import linen as nn
import folium
import jax
import jinja2
import matplotlib.pyplot as plt
from ml_collections import config_dict
import numpy as np
import pickle as pkl

from ithaca.eval import inference
from ithaca.models.model import Model
from ithaca.util.alphabet import GreekAlphabet

In [ ]:
#@title Configuration and auxiliary functions

class dataset_config:
  date_interval = 10
  date_max = 800
  date_min = -800

def get_subregion_name(id, region_map):
  return region_map['sub']['names_inv'][region_map['sub']['ids_inv'][id]]

def bce_ad(d):
  if d < 0:
    return f'{abs(d)} BCE'
  elif d > 0:
    return f'{abs(d)} AD'
  return 0


SALIENCY_SNIPPET_TEMPLATE = jinja2.Template("""
<div class="saliency">
  {% for char, score in pairs -%}
    <span
      style="background-color: rgba(171,71,188,{{'%.2f'|format(score)}});"
      title="Saliency score {{'%.2f'|format(score)}}">{{ char }}</span>
  {%- endfor %}
</div>
""")

SALIENCY_TEMPLATE = jinja2.Template("""<!DOCTYPE html>
<html>
<head>
<link rel="preconnect" href="https://fonts.googleapis.com">
<link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
<link href="https://fonts.googleapis.com/css2?family=Roboto+Mono:wght@400&family=Roboto:wght@400&display=swap" rel="stylesheet">
<style>
body { font-family: 'Roboto Mono', monospace; }
.saliency {
  word-wrap: break-word;
  white-space: normal;
}
</style>
</head>
<body>
{{body_html|safe}}
</body>
</html>
""")


def generate_saliency(text, saliency, snippet=False):
  """Generates saliency visualisation."""
  snippet_html = SALIENCY_SNIPPET_TEMPLATE.render(
      pairs=list(zip(text, saliency)))
  if snippet:
    return snippet_html
  return SALIENCY_TEMPLATE.render(body_html=snippet_html)


In [ ]:
#@title Load and create model
with open("checkpoint.pkl", "rb") as f:
  checkpoint = pkl.load(f)

model_config = config_dict.ConfigDict(checkpoint['model_config'])
params = jax.device_put(checkpoint['params'])

alphabet = GreekAlphabet()
alphabet.idx2word = checkpoint['alphabet']['idx2word']
alphabet.word2idx = checkpoint['alphabet']['word2idx']

vocab_char_size = checkpoint['model_config']['vocab_char_size']
vocab_word_size = checkpoint['model_config']['vocab_word_size']

region_map = checkpoint['region_map']

forward = functools.partial(Model(**model_config).apply, params)

In [ ]:
#@title  { run: "auto", vertical-output: true }
text = 'βασιλισσης και βασιλεως προσταξαντων αντι της προανακειμενης περι της αναθεσεως της προσευχης πλακος η υπογεγραμμενη επιγραφητω. βασιλευς πτολεμαιος ευεργετης την προσευχην ασυλον.' #@param {type:"string"}
assert 50 <= len(text) <= 750, "text should be between 50 and 750 chars long, got " + str(len(text))

In [ ]:
#@title Chronological attribution
#@markdown Ithaca’s chronological attribution hypotheses, visualized as a categorical distribution over decades, in yellow, between 800 BCE and 800 CE. This visualisation enables the handling of date intervals more effectively and aids the interpretability of the hypotheses.

attribution_results = inference.attribute(
    text=text,
    forward=forward,
    params=params,
    alphabet=alphabet,
    vocab_char_size=vocab_char_size,
    vocab_word_size=vocab_word_size,
    region_map=region_map
)

# Compute scores
date_pred_y = np.array(attribution_results.year_scores)
date_pred_x = np.arange(
  dataset_config.date_min + dataset_config.date_interval / 2,
  dataset_config.date_max + dataset_config.date_interval / 2,
  dataset_config.date_interval)
date_pred_argmax = date_pred_y.argmax(
) * dataset_config.date_interval + dataset_config.date_min + dataset_config.date_interval // 2
date_pred_avg = np.dot(date_pred_y, date_pred_x)

# Plot figure
fig = plt.figure(figsize=(10, 5), dpi=100)

plt.bar(date_pred_x, date_pred_y, color='#f2c852', width=10., label='Ithaca distribution')
plt.axvline(x=date_pred_avg, color='#67ac5b', linewidth=2., label='Ithaca average')


plt.ylabel('Probability', fontsize=14)
yticks = np.arange(0, 1.1, 0.1)
yticks_str = list(map(lambda x: f'{int(x*100)}%', yticks))
plt.yticks(yticks, yticks_str, fontsize=12, rotation=0)
plt.ylim(0, int((date_pred_y.max()+0.1)*10)/10)

plt.xlabel('Date', fontsize=14)
xticks = list(range(dataset_config.date_min, dataset_config.date_max + 1, 25))
xticks_str = list(map(bce_ad, xticks))
plt.xticks(xticks, xticks_str, fontsize=12, rotation=0)
plt.xlim(int(date_pred_avg - 100), int(date_pred_avg + 100))
plt.legend(loc='upper right', fontsize=12)

plt.show()

In [ ]:
#@title Chronological attribution saliency map
#@markdown Saliency map shows which unique input text features contributed the most to Ithaca’s top chronological attribution hypothesis.
display(HTML(generate_saliency(
    text=attribution_results.input_text,
    saliency=attribution_results.date_saliency)))

In [ ]:
import matplotlib.pyplot as plt
from nltk.util import bigrams
import numpy as np

# dictionary with 10 texts and target bigrams
texts_data = {
    "ID_219963": {
        "text": "βασιλισσης και βασιλεως προσταξαντων αντι της προανακειμενης περι της αναθεσεως της προσευχης πλακος η υπογεγραμμενη επιγραφητω. βασιλευς πτολεμαιος ευεργετης την προσευχην ασυλον.",
        "target_bigrams": [("βασιλευς", "πτολεμαιος")],
    },
    "ID_30483": {
        "text": "βιδεοι επι δαμοκλεους του του και φιλοκρατους ----ρι--τ- το β καλλιμαχος ---κλεους κλεων φιλοστρατος ---ιππου γαιος ιουλιος καλ-τικελω κασεν --------ρατου.",
        "target_bigrams": [("γαιος", "ιουλιος")],
    },
    "ID_28743": {
        "text": "λευκιος μομμιος λευκιου στρατηγος υπατος ρωμαιων απολλωνι ασκληπιωι υγιειαι.",
        "target_bigrams": [("στρατηγος", "υπατος"), ("λευκιος", "μομμιος")],
    },
    "ID_284303": {
        "text": "τιβεριος κλαυδιος τιβεριου κλαυδιου αγριππεινου υπατου υιος κυρεινα ----- τας εξεδρας τη γλυκυτατη πατριδι ανεθηκεν.",
        "target_bigrams": [("τιβεριος", "κλαυδιος")],
    },
    "ID_208503": {
        "text": "αυτοκρατορα καισαρα μαρκον αυρηλιον σευηρον αντωνεινον ανικητον ευσεβη ευτυχη σεβαστον παρθικον μεγιστον βρεταννικον μεγιστον αρχιερεα μεγιστον δημαρχικης εξουσιας πατερα πατριδος κουριεων η πολις.",
        "target_bigrams": [("ευσεβη", "ευτυχη")],
    },
    "ID_321813": {
        "text": "υριε αναπαυσον ιωαννην αζιζεου. κυριε ο θεος της αγιας μαριας και παντων των αγιων ελεησον παντος του κοσμου και βοηθεσον τους προσφεροντας και ιωαννην αναηλου του ευλαβεστατου διακονου μηνι ξανθικου ινδικτιωνος.",
        "target_bigrams": [("αγιας", "μαριας")],
    },
    "ID_216073": {
        "text": "υπερ βασιλισσα κλεοπατρα και βασιλευς πτολεμαιος θεων φιλομητορων και θεων σωτηρων ηρωδης. ------ και πανις δε-----------π--.",
        "target_bigrams": [("βασιλισσα", "κλεοπατρα"), ("βασιλευς", "πτολεμαιος")],
    },
    "ID_237533": {
        "text": "βασιλευς αντιοχος φιλοπαππος βασιλεως επιφανους του αντιοχου. βασιλευς αντιοχος βασιλεως αντιοχου. φιλοπαππος επιφανους βησαιευς. βασιλευς σελευκος αντιοχου νικατωρ.",
        "target_bigrams": [("βασιλευς", "σελευκος"), ("βασιλευς", "αντιοχος")],
    },
    "ID_342423": {
        "text": "μαρκος αυρηλιος μακεδων κατεσκευασα τον ταφον εμαυτω και βουλομαι ενταφηναι μετα την αποβιωσιν αλλον δε μηδενα ενεστιν η προμαμμην ------------ ενταφηναι και την γυναικα μου αυρηλιαν ο-----ολι δε μηδενα επει ο τολμησας υποκεισεται τω ιερωτατω ταμειω.",
        "target_bigrams": [("αρκος", "αυρηλιος"), ("μαρκος", "αυρηλιος")],
    },
    "ID_242543": {
        "text": "διι μαδβαχω και σελαμανει πατρωοις ευχην γαιος ουαλεριος προκλος και συμαχος των δειοκλους αμα των υων αυτων οικοδομησαντες εν τω ανατολικω μερι του περιβολου και μεσηνβρινην γωνιαν εκ των ιδιων εκτισαν δραχων α οικοδομησαν πηχεις μηκους μεν κ υψους δε πηχεις δια ----.",
        "target_bigrams": [("γαιος", "ουαλεριος")],
    },
}

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import rankdata

# Function to find bigram indices in the text
def find_bigram_indices(text, target_bigram):
    tokens = text.split()
    text_bigrams = [(tokens[i], tokens[i + 1]) for i in range(len(tokens) - 1)]

    try:
        bigram_index = text_bigrams.index(target_bigram)
        start_idx = sum(len(tokens[i]) + 1 for i in range(bigram_index))
        end_idx = start_idx + len(target_bigram[0]) + 1 + len(target_bigram[1])
        return start_idx, end_idx
    except ValueError:
        return None, None

# Function to compute AUC for a specific span
def compute_auc(start_idx, end_idx, saliency_map):
    return np.sum(saliency_map[start_idx:end_idx])

# Function to evaluate saliency distributions dynamically
def evaluate_saliency_distribution_dynamic(inference, texts_data, forward, params, alphabet, vocab_char_size, vocab_word_size, region_map, threshold_ratio=0.1):
    overall_results = {}

    for text_id, data in texts_data.items():
        text = data["text"]
        target_bigrams = data["target_bigrams"]

        attribution_results = inference.attribute(
            text=text,
            forward=forward,
            params=params,
            alphabet=alphabet,
            vocab_char_size=vocab_char_size,
            vocab_word_size=vocab_word_size,
            region_map=region_map
        )
        saliency_map = np.array(attribution_results.date_saliency)
        total_saliency = np.sum(saliency_map)

        bigram_results = []

        for bigram in target_bigrams:
            start_idx, end_idx = find_bigram_indices(text, bigram)
            if start_idx is None or end_idx is None:
                continue

            bigram_auc = compute_auc(start_idx, end_idx, saliency_map)
            overlap_saliency = bigram_auc / (total_saliency + 1e-10)
            top_peak_within = np.argmax(saliency_map) in range(start_idx, end_idx)

            bigram_results.append({
                "bigram": bigram,
                "bigram_auc": bigram_auc,
                "total_saliency": total_saliency,
                "overlap_saliency": overlap_saliency,
                "top_peak_within": top_peak_within,
            })

        auc_values = [result["bigram_auc"] for result in bigram_results]
        ranks = rankdata(-np.array(auc_values), method="min")

        ranked_results = [
            {**result, "rank": rank}
            for result, rank in zip(bigram_results, ranks)
        ]
        ranked_results = sorted(ranked_results, key=lambda x: x["rank"])

        relevant_indices = [idx for idx, result in enumerate(ranked_results) if result["overlap_saliency"] > threshold_ratio]

        mrr = sum(1 / (ranked_results[idx]["rank"] + 1) for idx in relevant_indices) / len(target_bigrams) if target_bigrams else 0
        average_precision = np.mean([(i + 1) / (ranked_results[idx]["rank"] + 1) for i, idx in enumerate(relevant_indices)]) if relevant_indices else 0
        ndcg = sum(1 / np.log2(ranked_results[idx]["rank"] + 2) for idx in relevant_indices) / len(target_bigrams) if target_bigrams else 0

        overall_results[text_id] = {
            "bigram_results": ranked_results,
            "mrr": mrr,
            "average_precision": average_precision,
            "ndcg": ndcg,
        }

    return overall_results

# Function to aggregate global metrics across all inscriptions
def aggregate_global_metrics(results):
    total_mrr = 0
    total_map = 0
    total_ndcg = 0
    total_bigrams = 0

    for text_id, result in results.items():
        total_mrr += result["mrr"]
        total_map += result["average_precision"]
        total_ndcg += result["ndcg"]
        total_bigrams += len(result["bigram_results"])

    avg_mrr = total_mrr / len(results)
    avg_map = total_map / len(results)
    avg_ndcg = total_ndcg / len(results)

    return {
        "Average MRR": avg_mrr,
        "Average MAP": avg_map,
        "Average NDCG": avg_ndcg,
        "Total Bigrams Evaluated": total_bigrams,
    }

# Function to aggregate AUC and overlap statistics
def aggregate_bigram_stats(results):
    all_auc_values = []
    all_overlap_saliency = []

    for result in results.values():
        for bigram_result in result["bigram_results"]:
            all_auc_values.append(bigram_result["bigram_auc"])
            all_overlap_saliency.append(bigram_result["overlap_saliency"])

    mean_auc = np.mean(all_auc_values)
    median_auc = np.median(all_auc_values)
    std_auc = np.std(all_auc_values)
    mean_overlap = np.mean(all_overlap_saliency)
    median_overlap = np.median(all_overlap_saliency)

    return {
        "Mean AUC": mean_auc,
        "Median AUC": median_auc,
        "AUC Std Dev": std_auc,
        "Mean Overlap Saliency": mean_overlap,
        "Median Overlap Saliency": median_overlap,
        "Total Bigrams": len(all_auc_values),
    }

# Function to plot AUC distribution
def plot_auc_distribution(auc_values):
    plt.figure(figsize=(8, 5))
    plt.hist(auc_values, bins=20, color="#69b3a2", edgecolor="black", alpha=0.7)
    plt.title("AUC Distribution Across All Bigrams")
    plt.xlabel("AUC Value")
    plt.ylabel("Frequency")
    plt.grid(alpha=0.3)
    plt.show()

# Function to plot overlap distribution
def plot_overlap_distribution(overlap_values):
    plt.figure(figsize=(8, 5))
    plt.hist(overlap_values, bins=20, color="#ffa07a", edgecolor="black", alpha=0.7)
    plt.title("Overlap Saliency Distribution Across All Bigrams")
    plt.xlabel("Overlap Saliency")
    plt.ylabel("Frequency")
    plt.grid(alpha=0.3)
    plt.show()

# Run evaluation
results = evaluate_saliency_distribution_dynamic(
    inference=inference,
    texts_data=texts_data,
    forward=forward,
    params=params,
    alphabet=alphabet,
    vocab_char_size=vocab_char_size,
    vocab_word_size=vocab_word_size,
    region_map=region_map
)

# Aggregate method-level statistics
global_metrics = aggregate_global_metrics(results)
bigram_stats = aggregate_bigram_stats(results)

# Print overall results
print("\n### Method-Level Evaluation Metrics ###")
for metric, value in global_metrics.items():
    print(f"{metric}: {value:.4f}")

print("\n### Bigram-Level AUC and Overlap Statistics ###")
for metric, value in bigram_stats.items():
    print(f"{metric}: {value:.4f}")

# Visualize distributions
all_auc_values = [bigram_result["bigram_auc"] for result in results.values() for bigram_result in result["bigram_results"]]
all_overlap_values = [bigram_result["overlap_saliency"] for result in results.values() for bigram_result in result["bigram_results"]]

plot_auc_distribution(all_auc_values)
plot_overlap_distribution(all_overlap_values)


#MRR: Εστιάζει μόνο στην πρώτη σωστή πρόβλεψη. Αν είναι μηδέν, σημαίνει ότι το σύστημα δεν εντόπισε κανένα σωστό bigram.
#MAP: Λαμβάνει υπόψη όλα τα σωστά bigrams, αλλά απαιτεί να είναι σωστά ταξινομημένα.
#NDCG: Είναι πιο ανεκτικό από το MAP, καθώς επιτρέπει στα σωστά bigrams να βρίσκονται και σε χαμηλότερες θέσεις, με μικρότερη "ποινή".


